<a href="https://colab.research.google.com/github/shirleyzz/cap5610_machineleanrning/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
import numpy as np
from keras import backend as K
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
#load the training and test data using keras
(x_train_origin, y_train_origin),(x_test_origin, y_test_origin) = mnist.load_data()

In [0]:
x_train = x_train_origin.reshape((60000, 28 * 28))
x_train = x_train.astype('float32') / 255

x_test = x_test_origin.reshape((10000, 28 * 28))
x_test = x_test.astype('float32') / 255

In [0]:
#split into 10 classes with categorial labels
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
y_train = to_categorical(y_train_origin, num_classes=10) 
y_test = to_categorical(y_test_origin, num_classes=10)

In [0]:
#calculate cross entropy
def compute_scores(w,b,X):
  return np.dot(X,w.T) + b
def softmax(scores):
  exp = np.exp(scores)
  sum_exp = np.sum(exp, axis=1,keepdims=True)
  softmax = exp / sum_exp
  return softmax
def cross_entropy(X, Y, scores):
  m = X.shape[0]
  loss = - (1 / m) * np.sum(Y * np.log(scores))
  return loss

In [0]:
#initialize w and b
def initialize_params_with_zeros(n_classes, n_features):
  w = np.zeros((n_classes, n_features))
  b = np.zeros((1, n_classes))
  return w,b

In [0]:
# predict
def predict(w,b,X):
  scores = compute_scores(w,b,X)
  probs = softmax(scores)
  preds = np.argmax(probs,axis=1)
  print(preds)
  #return np.argmax(probs, axis=1)[:, np.newaxis]
  return preds

In [0]:
#training process
def train(X_train, Y_train, n_iters=10, learning_rate=0.01,batch = 100, n_classes = 10):
  n_samples,n_features = X_train.shape
  w, b = initialize_params_with_zeros(n_classes,n_features)
  #print(w.shape)
  all_losses = []
  #stochastic gradient regression
  for item in range(n_iters):
    shuffled_indices = np.random.permutation(n_samples)
    X_shuffled = X_train[shuffled_indices,:]
    Y_shuffled = Y_train[shuffled_indices,:]
    
    for i in range(0, n_samples, batch):

      X_i = X_shuffled[i:i+batch,:]
      Y_i = Y_shuffled[i:i+batch,:]
      scores = compute_scores(w,b,X_i)
      probs = softmax(scores)
      loss = cross_entropy(X_i, Y_i, probs)
      #print(probs.shape)
      dw = -1. / n_samples * np.dot((Y_i - probs).T,X_i)
      db = -1. / n_samples * np.sum(Y_i - probs)
      w = w - learning_rate * dw
      b = b - learning_rate * db
    all_losses.append(loss)
    
    print(f'Iteration number: {item}, loss: {np.round(loss, 4)}')
  return w, b, all_losses

In [0]:
def model(X_train, Y_train, X_test, Y_test, num_iters, learning_rate,batch,n_classes):
  w, b, loss = train(X_train, Y_train, num_iters, learning_rate, batch,n_classes)
  #predict test/train set examples 
  Y_pred_test = predict(w, b, X_test)
  Y_pred_train = predict(w, b, X_train)
  #use argmax to return indices of the maximum values along an axis
  Y_true_test = np.argmax(Y_test, axis=1)
  Y_true_train = np.argmax(Y_train,axis=1)
                         
  #print train/test Errors
  print("")
  print("train accuracy: {} %".format(np.sum(Y_pred_train == Y_true_train)/len(Y_pred_train) * 100))
  print("test accuracy: {} %".format(np.sum(Y_pred_test == Y_true_test)/len(Y_pred_test) * 100))
 
  d = {"costs": loss, "Y_pred_test": Y_pred_test, 
        "Y_pred_train" : Y_pred_train, 
        "w" : w, 
        "b" : b,
        "learning_rate" : learning_rate,
        "num_iters": num_iters}
    
  return d

In [10]:
d = model(x_train, y_train, x_test, y_test, num_iters=10, learning_rate=0.01,batch=1000, n_classes=10)

Iteration number: 0, loss: 2.2919
Iteration number: 1, loss: 2.2824
Iteration number: 2, loss: 2.2696
Iteration number: 3, loss: 2.2601
Iteration number: 4, loss: 2.2491
Iteration number: 5, loss: 2.2378
Iteration number: 6, loss: 2.2289
Iteration number: 7, loss: 2.2169
Iteration number: 8, loss: 2.2103
Iteration number: 9, loss: 2.1974
[7 2 1 ... 8 8 6]
[3 0 4 ... 8 0 8]

train accuracy: 70.155 %
test accuracy: 70.47 %
